# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества F1 не меньше 0.75.

**Описание данных**

Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

## Загрузка и предобработка данных

In [1]:
import sys
!{sys.executable} -m pip install -q spacy
!{sys.executable} -m spacy download en

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import re
import warnings  # Импортируем все необходимые библиотеки для работы

import nltk
import numpy as np
import pandas as pd
import spacy
import torch
import transformers
from catboost import CatBoostClassifier
from nltk.corpus import stopwords as nltk_stopwords
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from tqdm import notebook
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ksiy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ksiy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# Загрузка данных
try:  
    df = pd.read_csv('/datasets/toxic_comments.csv')
    
except:
     df = pd.read_csv('toxic_comments.csv')

In [4]:
df.head()  # Вывод первых 5 строк

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()  # Вывод общей информации на экран

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
df['toxic'].value_counts()  # Соотношение классов

0    143106
1     16186
Name: toxic, dtype: int64

**Вывод** Имеем дисбаланс классов примерно 9 к 1.

В текстах необходимо оставить только латинские символы и пробелы. Воспользуемся функцией clear_text:

In [7]:
def clear_text(text):
    text2=re.sub(r'[^a-zA-Z ]',' ', text)
    return(" ".join(text2.split()))

In [8]:
%%time
df['clear'] = df['text'].apply(clear_text)
df['clear'] = df['clear'].str.lower()
df[['clear','text']]

CPU times: total: 2.73 s
Wall time: 2.72 s


,clear,text
0,explanation why the edits made under my userna...,Explanation\nWhy the edits made under my usern...
1,d aww he matches this background colour i m se...,D'aww! He matches this background colour I'm s...
2,hey man i m really not trying to edit war it s...,"Hey man, I'm really not trying to edit war. It..."
3,more i can t make any real suggestions on impr...,"""\nMore\nI can't make any real suggestions on ..."
4,you sir are my hero any chance you remember wh...,"You, sir, are my hero. Any chance you remember..."
...,...,...
159287,and for the second time of asking when your vi...,""":::::And for the second time of asking, when ..."
159288,you should be ashamed of yourself that is a ho...,You should be ashamed of yourself \n\nThat is ...
159289,spitzer umm theres no actual article for prost...,"Spitzer \n\nUmm, theres no actual article for ..."
159290,and it looks like it was actually you who put ...,And it looks like it was actually you who put ...


Для лемматизации всех текстов воспользуемся spacy:

In [9]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
sentence = "The striped bats are hanging on their feet for best"
doc = nlp(sentence)
" ".join([token.lemma_ for token in doc])

'the stripe bat be hang on their foot for good'

In [10]:
def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [11]:
%%time
df['lemm_text'] = df['clear'].apply(lemmatize)

CPU times: total: 17min 22s
Wall time: 17min 22s


In [12]:
print("*Первоначальный текст*:", df['clear'][:3])
print("*Преобразованный текст*:", df['lemm_text'][:3])

*Первоначальный текст*: 0    explanation why the edits made under my userna...
1    d aww he matches this background colour i m se...
2    hey man i m really not trying to edit war it s...
Name: clear, dtype: object
*Преобразованный текст*: 0    explanation why the edit make under my usernam...
1    d aww he match this background colour I m seem...
2    hey man I m really not try to edit war it s ju...
Name: lemm_text, dtype: object


In [13]:
features = df['lemm_text']  # Выделим признаки
target = df['toxic']  # Целевой признак

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=123
)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=123
)

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ksiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Вычислим TF-IDF для корпуса текстов:

## Обучение моделей

### Логистическая регрессия

In [15]:
pipeline_lr = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("lr", LogisticRegression(random_state=123)),
    ]
)

Ввиду сильного дисбаланса классов улучшать метрику F1 будем перевобором порога классификации:

In [16]:
pipeline_lr.fit(features_train, target_train)
probabilities = pipeline_lr.predict_proba(features_valid)
probabilities_one = probabilities[:, 1]

for threshold in np.arange(0.4, 0.9, 0.02):
    predicted_v = probabilities_one > threshold
    precision = precision_score(target_valid, predicted_v)
    recall = recall_score(target_valid, predicted_v) 
    f1 = f1_score(target_valid, predicted_v)
    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}, F1 = {:.3f}".format(
        threshold, precision, recall, f1))

Порог = 0.40 | Точность = 0.896, Полнота = 0.664, F1 = 0.763
Порог = 0.42 | Точность = 0.906, Полнота = 0.650, F1 = 0.757
Порог = 0.44 | Точность = 0.915, Полнота = 0.637, F1 = 0.751
Порог = 0.46 | Точность = 0.921, Полнота = 0.627, F1 = 0.746
Порог = 0.48 | Точность = 0.927, Полнота = 0.617, F1 = 0.741
Порог = 0.50 | Точность = 0.934, Полнота = 0.606, F1 = 0.735
Порог = 0.52 | Точность = 0.939, Полнота = 0.593, F1 = 0.727
Порог = 0.54 | Точность = 0.944, Полнота = 0.581, F1 = 0.719
Порог = 0.56 | Точность = 0.948, Полнота = 0.568, F1 = 0.711
Порог = 0.58 | Точность = 0.949, Полнота = 0.559, F1 = 0.703
Порог = 0.60 | Точность = 0.953, Полнота = 0.551, F1 = 0.698
Порог = 0.62 | Точность = 0.954, Полнота = 0.540, F1 = 0.690
Порог = 0.64 | Точность = 0.956, Полнота = 0.532, F1 = 0.684
Порог = 0.66 | Точность = 0.960, Полнота = 0.522, F1 = 0.676
Порог = 0.68 | Точность = 0.961, Полнота = 0.511, F1 = 0.667
Порог = 0.70 | Точность = 0.962, Полнота = 0.502, F1 = 0.660
Порог = 0.72 | Точность 

Исходя из полученных данных, примем порог классификации 0.40:

In [17]:
predicted_valid = (pipeline_lr.predict_proba(features_valid)[:,1] > 0.40)*1
f1_log_r_1 = f1_score(target_valid, predicted_valid).round(4)
print('Метрика F1 качества предсказания модели LogisticRegression с порогом классификации 0,40 равна : ',f1_log_r_1)

Метрика F1 качества предсказания модели LogisticRegression с порогом классификации 0,40 равна :  0.7625


### svm. LinearSVC

In [18]:
pipeline_svc = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("lsvc", svm.LinearSVC(random_state=123)),
    ]
)

In [19]:
%%time
pipeline_svc.fit(features_train, target_train)
score_svc = cross_val_score(pipeline_svc, features_train, target_train, cv=3, scoring='f1').mean()
print('Метрика F1 качества предсказания модели LinearSVC равна : ',score_svc.round(4))

Метрика F1 качества предсказания модели LinearSVC равна :  0.7751
CPU times: total: 21.9 s
Wall time: 21.9 s


**Вывод** Обе модели показали метрики, соответствующие заданию.

## Обучение с BERT

In [20]:
df_bert = df.sample(1000)  # Для обучения модели возьмем 3000 строк

In [21]:
df_bert['toxic'].value_counts()  # Проверяем баланс классов

0    910
1     90
Name: toxic, dtype: int64

In [22]:
features_bert = df_bert['text']  # Выделяем признаки
target_bert = df_bert['toxic']  # Целевой признак
features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(
    features_bert, target_bert, stratify=df_bert['toxic'], test_size=0.5, random_state=1234
)  # Размер тренировочной и тестовой выборок возьмем 50/50

In [23]:
model = transformers.BertModel.from_pretrained(
    "unitary/toxic-bert")  # Инициализируем модель Bert
tokenizer = transformers.BertTokenizer.from_pretrained(
    "unitary/toxic-bert")


def tokenize(features): # Функция разделения текста на токены и кодирования

    tokenized = features.apply(
        lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)

    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
    
    return padded

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
def embeddings(padded):  # Функция перевода предобработанных токенов в векторы
    
    attention_mask = np.where(padded != 0, 1, 0)
    
    batch_size = 100
    embeddings = []
    for i in tqdm(range(padded.shape[0] // batch_size)):
            batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
            attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

            with torch.no_grad():
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)

            embeddings.append(batch_embeddings[0][:,0,:].numpy())
    return embeddings

In [25]:
padded_train = tokenize(features_train_bert)
padded_test = tokenize(features_test_bert)

In [26]:
embeddings_train = embeddings(padded_train)
embeddings_test = embeddings(padded_test)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
features_train2 = np.concatenate(embeddings_train)
features_test2 = np.concatenate(embeddings_test)

## Обучение моделей

### Логистическая регрессия

В этот раз попробуем учесть дисбаланс классов с помощью параметра class_weight:

In [28]:
model = LogisticRegression(class_weight='balanced', random_state=1234)

score = cross_val_score(model, features_train2, target_train_bert, cv=3, scoring='f1').mean()
print("Метрика на тренировочной выборке F1  = %.4f" % score)

Метрика на тренировочной выборке F1  = 0.9146


### CatBoostClassifier

In [29]:
%%time
model_c = CatBoostClassifier(
    iterations=1000,
    verbose=100,
    random_state=1234
) 
score_c = cross_val_score(model_c, features_train2, target_train_bert, cv=3, scoring='f1').mean()

Learning rate set to 0.006442
0:	learn: 0.6792423	total: 148ms	remaining: 2m 28s
100:	learn: 0.1242228	total: 7.98s	remaining: 1m 11s
200:	learn: 0.0394755	total: 15.8s	remaining: 1m 2s
300:	learn: 0.0186161	total: 23.6s	remaining: 54.7s
400:	learn: 0.0110520	total: 31.3s	remaining: 46.8s
500:	learn: 0.0075789	total: 39.1s	remaining: 38.9s
600:	learn: 0.0056321	total: 46.9s	remaining: 31.1s
700:	learn: 0.0044147	total: 54.6s	remaining: 23.3s
800:	learn: 0.0036110	total: 1m 2s	remaining: 15.5s
900:	learn: 0.0030328	total: 1m 10s	remaining: 7.71s
999:	learn: 0.0026091	total: 1m 17s	remaining: 0us
Learning rate set to 0.006442
0:	learn: 0.6795537	total: 85.5ms	remaining: 1m 25s
100:	learn: 0.1312475	total: 7.92s	remaining: 1m 10s
200:	learn: 0.0414920	total: 16.1s	remaining: 1m 4s
300:	learn: 0.0198466	total: 24.2s	remaining: 56.1s
400:	learn: 0.0119154	total: 32.1s	remaining: 47.9s
500:	learn: 0.0081035	total: 40.2s	remaining: 40.1s
600:	learn: 0.0060010	total: 48.3s	remaining: 32.1s
700

In [30]:
print("Метрика на тренировочной выборке F1 = %.4f" % score_c)

Метрика на тренировочной выборке F1 = 0.8419


**Вывод**  За лучшую модель примем модель логистической регрессии, обученной на данных, преобразованных с помощью BERT, c метрикой F1 = 0.9146

## Тестирование модели

In [32]:
model.fit(features_train2, target_train_bert)
pred = model.predict(features_test2)
print("Тестовая метрика F1 финальной модели: ", f1_score(target_test_bert, pred).round(2))

Тестовая метрика F1 финальной модели:  0.93


## Вывод

В результате работы был проведен анализ входных данных, выявлен дисбаланс классов. Задача решалась двумя способами: обработкой и лемматизацией текстов с помощью spaCy и предобработкой текстов с помощью библиотеки transformers и BERT-модели. В каждом из вариантов данные были разделены на тренировочную и тестовую выборки, затем обучены модели. Дисбаланс классов учитывался в модели логистической регрессии с помощью подбора порога классификации и указанием параметра class_weight='balanced'.\
Наилучшую метрику на валидационной выборке показала модель LogisticRegression с предобработкой текстов с помощью BERT: F1 = 0.9146.\
Затем, модель была протестирована на тестовой выборке.\
**На тестовой выборке модель LogisticRegression показала результат F1 = 0.93**, что соответствует заданию задачи.